In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto based on the postalcode and borough informationa

## Toronto Dataframe

In [77]:
#importing libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
  - defaults/noarch::ibm-wsrt-py37main-main==custom=1976
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=1976
done

# All requested packages already installed.



In [78]:
#Wikipedia Scrapping
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html=requests.get(url).text
soup = BeautifulSoup(html,'html5lib')

In [79]:
#Creating Dataframe #1 from web scrapping
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [80]:
df.shape

(103, 3)

In [81]:
#Dataframe #2 grouping Neighborhood by Postal Code
df_postcode = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df_postcode.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [82]:
#Dataframe #3 adding geographical coordinates of each postal code
lat_long_df = pd.read_csv('https://cocl.us/Geospatial_data', index_col='Postal Code')
df_Toronto = df_postcode.join(lat_long_df, on='PostalCode')
df_Toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [83]:
df_Toronto.shape

(103, 5)

In [109]:
#Visualization of Neighborhoods in Toronto
latitude= 43.6511
longitude=-79.38
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_Toronto['Latitude'], df_Toronto['Longitude'], df_Toronto['Borough'], df_Toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [135]:
#Let's analize only Neighborhoods with the word Toronto
df_Toronto2=df_Toronto[df_Toronto['Neighborhood'].astype(str).str.contains('Toronto')]
df_Toronto2.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
59,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752
60,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576
66,M5S,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049
88,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321


In [134]:
df_Toronto2.shape

(5, 5)

In [121]:
#Visualization of Neighborhoods in Toronto with the word Toronto
latitude= 43.6511
longitude=-79.38
map_Toronto2 = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_Toronto2['Latitude'], df_Toronto2['Longitude'], df_Toronto2['Borough'], df_Toronto2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto2)  
    
map_Toronto2

## Dataframes with venues

In [131]:
#Dataframe #4 Toronto Venues
nearby_venues = pd.read_json("https://raw.githubusercontent.com/ibm-developer-skills-network/yczvh-DataFilesForIBMProjects/master/segmenting_neighborhoods.json")    
nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                 'Venue', 
                 'Venue Latitude', 
                 'Venue Longitude', 
                 'Venue Category']
Toronto_venues=nearby_venues
Toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant


In [132]:
Toronto_venues.shape

(1337, 7)

## One hot encoding

In [124]:
# one hot encoding
Toronto_onehot1 = pd.get_dummies(Bars_venues[['Venue Category']], prefix="", prefix_sep="")
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Harbourfront East, Union Station, Toronto Islands",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.033333,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
1,"New Toronto, Mimico South, Humber Bay Shores",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083333,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
2,North Toronto West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.052632
3,"Toronto Dominion Centre, Design Exchange",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,...,0.033333,0.0,0.0,0.000000,0.0,0.0,0.033333,0.0,0.0,0.000000
4,"University of Toronto, Harbord",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.033333,0.0,0.0,0.000000,0.0,0.0,0.033333


In [125]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Harbourfront East, Union Station, Toronto Islands",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.033333,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
1,"New Toronto, Mimico South, Humber Bay Shores",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083333,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
2,North Toronto West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.052632
3,"Toronto Dominion Centre, Design Exchange",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,...,0.033333,0.0,0.0,0.000000,0.0,0.0,0.033333,0.0,0.0,0.000000
4,"University of Toronto, Harbord",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.033333,0.0,0.0,0.000000,0.0,0.0,0.033333


In [126]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [141]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Harbourfront East, Union Station, Toronto Islands",Plaza,Park,Hotel,Café,New American Restaurant,Italian Restaurant,Bistro,Basketball Stadium,Lake,Salad Place
1,"New Toronto, Mimico South, Humber Bay Shores",Gym,American Restaurant,Liquor Store,Bakery,Café,Seafood Restaurant,Restaurant,Fast Food Restaurant,Pizza Place,Pet Store
2,North Toronto West,Clothing Store,Coffee Shop,Yoga Studio,Sporting Goods Shop,Fast Food Restaurant,Mexican Restaurant,Shoe Store,Seafood Restaurant,Café,Salon / Barbershop
3,"Toronto Dominion Centre, Design Exchange",Café,Coffee Shop,Restaurant,Bakery,Japanese Restaurant,Train Station,Steakhouse,Beer Bar,Museum,Sandwich Place
4,"University of Toronto, Harbord",Café,Bar,Bakery,Bookstore,Italian Restaurant,Japanese Restaurant,Yoga Studio,Restaurant,Sandwich Place,French Restaurant


## Clustering

In [142]:
#I want to group Toronto's Neighborhoods by 
# set number of clusters
kclusters = 2
Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 0, 0, 0], dtype=int32)

In [143]:
Toronto_merged = df_Toronto2
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# merge Toronto_grouped with df_Toronto to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() 

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0,Clothing Store,Coffee Shop,Yoga Studio,Sporting Goods Shop,Fast Food Restaurant,Mexican Restaurant,Shoe Store,Seafood Restaurant,Café,Salon / Barbershop
59,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,0,Plaza,Park,Hotel,Café,New American Restaurant,Italian Restaurant,Bistro,Basketball Stadium,Lake,Salad Place
60,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,0,Café,Coffee Shop,Restaurant,Bakery,Japanese Restaurant,Train Station,Steakhouse,Beer Bar,Museum,Sandwich Place
66,M5S,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049,0,Café,Bar,Bakery,Bookstore,Italian Restaurant,Japanese Restaurant,Yoga Studio,Restaurant,Sandwich Place,French Restaurant
88,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,1,Gym,American Restaurant,Liquor Store,Bakery,Café,Seafood Restaurant,Restaurant,Fast Food Restaurant,Pizza Place,Pet Store


In [144]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [146]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
46,Central Toronto,0,Clothing Store,Coffee Shop,Yoga Studio,Sporting Goods Shop,Fast Food Restaurant,Mexican Restaurant,Shoe Store,Seafood Restaurant,Café,Salon / Barbershop
59,Downtown Toronto,0,Plaza,Park,Hotel,Café,New American Restaurant,Italian Restaurant,Bistro,Basketball Stadium,Lake,Salad Place
60,Downtown Toronto,0,Café,Coffee Shop,Restaurant,Bakery,Japanese Restaurant,Train Station,Steakhouse,Beer Bar,Museum,Sandwich Place
66,Downtown Toronto,0,Café,Bar,Bakery,Bookstore,Italian Restaurant,Japanese Restaurant,Yoga Studio,Restaurant,Sandwich Place,French Restaurant


In [147]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
88,Etobicoke,1,Gym,American Restaurant,Liquor Store,Bakery,Café,Seafood Restaurant,Restaurant,Fast Food Restaurant,Pizza Place,Pet Store
